In [ ]:
!pip install twilio requests pandas gspread oauth2client python-telegram-bot tweepy spacy
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 58.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install transformers
!pip install twilio requests pandas gspread oauth2client python-telegram-bot tweepy spacy
!python -m spacy download en_core_web_sm
import spacy
import gspread
from oauth2client.service_account import ServiceAccountCredentials


# Load English NLP model
nlp = spacy.load("en_core_web_sm")

# Example social media messages
messages = [
    "OMG! A huge fire has broken out near Chennai Central railway station. People, stay safe! 🚒🔥 #FireAccident #Chennai",
    "Breaking News: A massive earthquake just hit Delhi NCR. Buildings shaking! #Earthquake #DelhiNCR",
    "Severe flooding reported in T. Nagar. Roads are submerged, avoid the area.",
    "Gas leak detected in an industrial plant near Guindy. Emergency services on site.",
    "Landslide near Ooty has blocked major roads. Travel delayed! #Landslide",
    "Tsunami warning issued for the Chennai coast. Evacuate immediately!",
    "A major accident on the Mumbai-Pune expressway has caused heavy traffic. Drive safe!"
]

# Emergency keywords
EMERGENCY_KEYWORDS = {
    "fire": "Fire",
    "earthquake": "Earthquake",
    "accident": "Accident",
    "landslide": "Landslide",
    "tsunami": "Tsunami",
    "flood": "Flood"
}

# Import the summarization pipeline from transformers
from transformers import pipeline
summarizer = pipeline("summarization")

# Function to extract emergency alerts
def extract_alerts(messages):
    alerts = []
    for msg in messages:
        doc = nlp(msg)
        detected_type = None

        for keyword, event_type in EMERGENCY_KEYWORDS.items():
            if keyword in msg.lower():
                detected_type = event_type
                break  # Stop checking once a keyword matches

        if detected_type:
            location = "Unknown"
            for ent in doc.ents:
                if ent.label_ in ["GPE", "LOC"]:  # Detect locations (cities, regions)
                    location = ent.text
                    break

            alert = {
                "message": msg,
                "location": location,
                "type": detected_type
            }
            alerts.append(alert)
    return alerts

# Extract emergency alerts
alerts = extract_alerts(messages)

# ✅ Make sure to replace 'your_json_file_path.json' with the actual path to your credentials file
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
file_path = "# Replace with your JSON file path if it is in a different location" # Replace with your JSON file path if it is in a different location
creds = ServiceAccountCredentials.from_json_keyfile_name(file_path, scope)
client = gspread.authorize(creds)

# Open the Google Sheet
sheet = client.open("EMERGENCY ALERT PROJECT").sheet1  # ✅ Inserted your sheet name here

# Store alerts in Google Sheets
for alert in alerts:
    summary = summarizer(alert["message"], max_length=20, min_length=10, do_sample=False)[0]['summary_text']
    sheet.append_row([alert["type"], alert["location"], summary])  # Now 'sheet' is defined
    print(f"✅ Stored Alert: {alert['type']} in {alert['location']} - {summary}")

print("🚀 Google Sheets updated with emergency alerts!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 673.5/673.5 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


✅ Stored Alert: Fire in Chennai Central -  A huge fire has broken out near Chennai Central railway station . People, stay safe!
✅ Stored Alert: Earthquake in Unknown -  A massive earthquake just hit Delhi NCR. Buildings shaking! #Earthquake #
✅ Stored Alert: Flood in T. Nagar -  Roads are submerged, avoid the area . Severe flooding reported in T. Nagar


Your max_length is set to 20, but your input_length is only 17. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


✅ Stored Alert: Landslide in Unknown -  Landslide near Ooty has blocked major roads . Travel delayed . Landsl
✅ Stored Alert: Tsunami in Tsunami -  Tsunami warning issued for the Chennai coast . Evacuate immediately! Evac
✅ Stored Alert: Accident in Mumbai -  A major accident on the Mumbai-Pune expressway has caused heavy traffic . A
🚀 Google Sheets updated with emergency alerts!


In [ ]:
import os
print(os.path.exists("# Replace with your JSON file path if it is in a different location"))


True


In [ ]:
!pip install tweepy


In [ ]:
import tweepy

# Twitter API credentials
API_KEY = "XXXXXXX"
API_SECRET_KEY = "YYYYYYYYYY"
ACCESS_TOKEN = "FFFFFFFFFFFFFFFFFFFFFFFFFFF"
ACCESS_TOKEN_SECRET = "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB"



# Authenticate with Twitter
auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Create API object
api = tweepy.API(auth)

# Test API connection
try:
    user = api.verify_credentials()
    print("Authentication successful! Logged in as:", user.screen_name)
except:
    print("Authentication failed! Please check your credentials.")



Authentication successful! Logged in as: sanjay221501158


In [ ]:
!pip install tweepy pandas spacy flask
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 103.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install streamlit tweepy gspread oauth2client transformers spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00


In [ ]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 100.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# --- Install & Imports ---
!pip install --upgrade torch torchvision torchaudio transformers gspread oauth2client spacy
!python -m spacy download en_core_web_sm

import tweepy
import gspread
import spacy
import time
import re
from datetime import datetime
from oauth2client.service_account import ServiceAccountCredentials
from transformers import pipeline

# --- Load NLP Models ---
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
summarizer = pipeline("summarization")
nlp = spacy.load("en_core_web_sm")

# --- Disaster Labels & Keywords ---
CANDIDATE_LABELS = ["accident", "fire", "earthquake", "landslide", "tsunami", "flood"]

DISASTER_KEYWORDS = {
    "fire": ["blaze", "burning", "flames", "wildfire", "smoke", "firefighters"],
    "accident": ["collision", "crash", "injured", "killed", "fatal", "ambulance"],
    "earthquake": ["richter", "tremor", "aftershock", "magnitude", "seismic"],
    "landslide": ["mudslide", "debris", "collapsed", "hill", "mountain", "buried"],
    "tsunami": ["wave", "seawater", "ocean", "quake", "evacuation"],
    "flood": ["rain", "overflow", "submerged", "drowned", "inundated", "storm"]
}

EMERGENCY_HASHTAGS = {
    "#fire": "Fire",
    "#earthquake": "Earthquake",
    "#accident": "Accident",
    "#landslide": "Landslide",
    "#tsunami": "Tsunami",
    "#flood": "Flood"
}

NEGATIVE_CONTEXT_KEYWORDS = [
    "song", "music", "album", "mixtape", "dance", "rap", "party", "track",
    "beat", "concert", "reels", "vibe", "chill", "joint", "smoke", "420", "weed",
    "cigarette", "hookah", "dj", "soundcloud", "studio"
]

# Twitter API credentials
API_KEY = "XXXXXXX"
API_SECRET_KEY = "YYYYYYYYYY"
ACCESS_TOKEN = "FFFFFFFFFFFFFFFFFFFFFFFFFFF"
ACCESS_TOKEN_SECRET = "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA"

# --- Google Sheets Credentials ---
GOOGLE_SHEET_JSON_PATH = "# Replace with your JSON file path if it is in a different location"
SHEET_NAME = "TWITTER EMERGENCY ALERTS"

# --- Initialize Twitter and Google Clients ---
client = tweepy.Client(bearer_token=BEARER_TOKEN)
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_SHEET_JSON_PATH, scope)
sheet = gspread.authorize(creds).open(SHEET_NAME).sheet1

# --- Fetch Tweets Function ---
def fetch_tweets_v2(hashtag, count=10):
    query = f"{hashtag} -is:retweet lang:en"
    try:
        response = client.search_recent_tweets(query=query, max_results=count, tweet_fields=["text", "author_id", "created_at", "id"])
        return [{
            "text": tweet.text,
            "user": tweet.author_id,
            "id": tweet.id
        } for tweet in response.data] if response.data else []
    except tweepy.TooManyRequests:
        print(f"🚨 Rate limit hit for '{hashtag}'. Waiting 60 seconds...")
        time.sleep(60)
        return fetch_tweets_v2(hashtag, count)
    except Exception as e:
        print(f"❌ Error fetching tweets for '{hashtag}':", e)
        return []

# --- Extract Valid Alerts ---
def extract_alerts(tweets, expected_type):
    alerts = []
    keywords = DISASTER_KEYWORDS[expected_type.lower()]
    for tweet in tweets:
        msg = tweet["text"]
        msg_lower = msg.lower()

        if any(neg in msg_lower for neg in NEGATIVE_CONTEXT_KEYWORDS):
            continue

        classification = classifier(msg, CANDIDATE_LABELS)
        top_label = classification["labels"][0]
        confidence = classification["scores"][0]

        if top_label.lower() != expected_type.lower() or confidence < 0.75:
            continue

        if not any(re.search(rf"\b{k}\b", msg_lower) for k in keywords):
            continue

        doc = nlp(msg)
        location = next((ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC"]), "Unknown")

        alerts.append({
            "message": msg,
            "location": location,
            "type": expected_type,
            "user": tweet["user"],
            "url": f"https://twitter.com/i/web/status/{tweet['id']}"
        })
    return alerts

# --- Main Script to Run ---
def run_emergency_check(test_hashtag):
    test_type = EMERGENCY_HASHTAGS.get(test_hashtag.lower())
    if not test_type:
        print("❌ Invalid hashtag provided.")
        return

    print(f"🔍 Searching tweets for: {test_hashtag}")
    tweets = fetch_tweets_v2(test_hashtag, count=10)

    if not tweets:
        print("❌ No tweets found.")
        return

    print(f"✅ {len(tweets)} tweets fetched. Filtering relevant alerts...")
    all_alerts = extract_alerts(tweets, test_type)

    if not all_alerts:
        print("⚠️ No relevant disaster alerts found.")
        return

    for alert in all_alerts:
        print("⚠️ Alert Detected:")
        print(f"Type: {alert['type']}, Location: {alert['location']}")
        print(f"Tweet: {alert['message']}")
        print(f"Link: {alert['url']}")
        print("-" * 80)

        try:
            summary = summarizer(alert["message"], max_length=20, min_length=10, do_sample=False)[0]["summary_text"]
        except:
            summary = alert["message"][:100]

        try:
            sheet.append_row([
                alert["type"],
                alert["location"],
                str(alert["user"]),
                summary,
                alert["url"],
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ])
            print(f"✅ Alert stored: {summary}")
        except Exception as e:
            print(f"❌ Failed to write to sheet: {e}")

    print("🚀 Processing complete.")

# --- Run Example ---
run_emergency_check("#flood")  # Change hashtag as needed

# --- Run for All Emergency Hashtags (LIMITED API USE) ---
#from datetime import datetime

"""def run_all_emergency_checks():
    for hashtag in EMERGENCY_HASHTAGS.keys():
        try:
            print("\n" + "=" * 80)
            print(f"🔍 Checking for: {hashtag}")

            # Fetch only 1 tweet to save API usage
            tweets = fetch_tweets_v2(hashtag, count=1)

            if not tweets:
                print("❌ No tweets found.")
                continue

            expected_type = EMERGENCY_HASHTAGS[hashtag.lower()]
            alerts = extract_alerts(tweets, expected_type)

            if not alerts:
                print("⚠️ No relevant alerts found.")
                continue

            for alert in alerts:
                print("⚠️ Alert Detected:")
                print(f"Type: {alert['type']}, Location: {alert['location']}")
                print(f"Tweet: {alert['message']}")
                print(f"Link: {alert['url']}")

                try:
                    summary = summarizer(alert["message"], max_length=20, min_length=10, do_sample=False)[0]["summary_text"]
                except Exception as e:
                    print(f"⚠️ Summarization failed: {e}")
                    summary = alert["message"][:100]

                try:
                    sheet.append_row([
                        alert["type"],
                        alert["location"],
                        str(alert["user"]),
                        summary,
                        alert["url"],
                        datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    ])
                    print(f"✅ Alert stored successfully.")
                except Exception as e:
                    print(f"❌ Failed to write to sheet: {e}")
        except Exception as main_error:
            print(f"🚨 Unexpected error for hashtag {hashtag}: {main_error}")"""



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


🔍 Searching tweets for: #flood
✅ 10 tweets fetched. Filtering relevant alerts...
⚠️ Alert Detected:
Type: Flood, Location: Mogadishu
Tweet: Seven dead after heavy rain hits Mogadishu
https://t.co/A9VKjIdzsL

#dead
#heavy
#rain
#hit
#Mogadishu
#Somalia
#flood
Link: https://twitter.com/i/web/status/1921421796403789913
--------------------------------------------------------------------------------
✅ Alert stored:  Seven dead after heavy rain hits Mogadishu . Heavy rain kills seven people in Mog
⚠️ Alert Detected:
Type: Flood, Location: Texas
Tweet: Heavy rains hammered parts of #Texas claiming at least one life

VC: Chris G 

6/7

#Flood #TXwx #USwx #Flooding #Flashflood #Rain #Weather #Viral #Climate https://t.co/ZTApXef2dI
Link: https://twitter.com/i/web/status/1921302915211268432
--------------------------------------------------------------------------------
✅ Alert stored:  Heavy rains hammered parts of #Texas claiming at least one life . Heavy rains hit parts
⚠️ Alert Detected:
Typ

In [ ]:
# --- Install & Imports ---
!pip install --upgrade torch torchvision torchaudio transformers gspread oauth2client spacy
!python -m spacy download en_core_web_sm

import tweepy
import gspread
import spacy
import time
import re
from datetime import datetime
from oauth2client.service_account import ServiceAccountCredentials
from transformers import pipeline

# --- Load NLP Models ---
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
summarizer = pipeline("summarization")
nlp = spacy.load("en_core_web_sm")

# --- Disaster Labels & Keywords ---
CANDIDATE_LABELS = ["accident", "fire", "earthquake", "landslide", "tsunami", "flood"]

DISASTER_KEYWORDS = {
    "fire": ["blaze", "burning", "flames", "wildfire", "smoke", "firefighters"],
    "accident": ["collision", "crash", "injured", "killed", "fatal", "ambulance"],
    "earthquake": ["richter", "tremor", "aftershock", "magnitude", "seismic"],
    "landslide": ["mudslide", "debris", "collapsed", "hill", "mountain", "buried"],
    "tsunami": ["wave", "seawater", "ocean", "quake", "evacuation"],
    "flood": ["rain", "overflow", "submerged", "drowned", "inundated", "storm"]
}

EMERGENCY_HASHTAGS = {
    "#fire": "Fire",
    "#earthquake": "Earthquake",
    "#accident": "Accident",
    "#landslide": "Landslide",
    "#tsunami": "Tsunami",
    "#flood": "Flood"
}

NEGATIVE_CONTEXT_KEYWORDS = [
    "song", "music", "album", "mixtape", "dance", "rap", "party", "track",
    "beat", "concert", "reels", "vibe", "chill", "joint", "smoke", "420", "weed",
    "cigarette", "hookah", "dj", "soundcloud", "studio"
]

# Twitter API credentials
API_KEY = "NmAtlRDhpsWVMXL03tfy8dGgx"
API_SECRET_KEY = "1iWUmP9WlGpCDo0ELagelfNeM6uyK91oFfC6lHmttbYsrVG0s9"
ACCESS_TOKEN = "1921411850429923328-TvE4KKZWuIwifPQUH2WkwdOlZH2PWT"
ACCESS_TOKEN_SECRET = "vdWXSWUG9xVTgsX8rLyMM0IZRgl0GNI4ZQM8DW7wel8wn"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPq81AEAAAAAl8XbdvHONQb9F61bXqbNk1JeI14%3D0thb5xzDZTXdbT1hnewms1zxHwRld2gSnqMtfUz0XW80jTOfx0"

# --- Google Sheets Credentials ---
GOOGLE_SHEET_JSON_PATH = "/content/twitter-emergncy-project-3789f10635d0.json"
SHEET_NAME = "TWITTER EMERGENCY ALERTS"

# --- Initialize Twitter and Google Clients ---
client = tweepy.Client(bearer_token=BEARER_TOKEN)
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_SHEET_JSON_PATH, scope)
sheet = gspread.authorize(creds).open(SHEET_NAME).sheet1

# --- Fetch Tweets Function ---
def fetch_tweets_v2(hashtag, count=10):
    query = f"{hashtag} -is:retweet lang:en"
    try:
        response = client.search_recent_tweets(query=query, max_results=count, tweet_fields=["text", "author_id", "created_at", "id"])
        return [{
            "text": tweet.text,
            "user": tweet.author_id,
            "id": tweet.id
        } for tweet in response.data] if response.data else []
    except tweepy.TooManyRequests:
        print(f"🚨 Rate limit hit for '{hashtag}'. Waiting 60 seconds...")
        time.sleep(60)
        return fetch_tweets_v2(hashtag, count)
    except Exception as e:
        print(f"❌ Error fetching tweets for '{hashtag}':", e)
        return []

# --- Extract Valid Alerts ---
def extract_alerts(tweets, expected_type):
    alerts = []
    keywords = DISASTER_KEYWORDS[expected_type.lower()]
    for tweet in tweets:
        msg = tweet["text"]
        msg_lower = msg.lower()

        if any(neg in msg_lower for neg in NEGATIVE_CONTEXT_KEYWORDS):
            continue

        classification = classifier(msg, CANDIDATE_LABELS)
        top_label = classification["labels"][0]
        confidence = classification["scores"][0]

        if top_label.lower() != expected_type.lower() or confidence < 0.75:
            continue

        if not any(re.search(rf"\b{k}\b", msg_lower) for k in keywords):
            continue

        doc = nlp(msg)
        location = next((ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC"]), "Unknown")

        alerts.append({
            "message": msg,
            "location": location,
            "type": expected_type,
            "user": tweet["user"],
            "url": f"https://twitter.com/i/web/status/{tweet['id']}"
        })
    return alerts

# --- Main Script to Run ---
def run_emergency_check(test_hashtag):
    test_type = EMERGENCY_HASHTAGS.get(test_hashtag.lower())
    if not test_type:
        print("❌ Invalid hashtag provided.")
        return

    print(f"🔍 Searching tweets for: {test_hashtag}")
    tweets = fetch_tweets_v2(test_hashtag, count=10)

    if not tweets:
        print("❌ No tweets found.")
        return

    print(f"✅ {len(tweets)} tweets fetched. Filtering relevant alerts...")
    all_alerts = extract_alerts(tweets, test_type)

    if not all_alerts:
        print("⚠️ No relevant disaster alerts found.")
        return

    for alert in all_alerts:
        print("⚠️ Alert Detected:")
        print(f"Type: {alert['type']}, Location: {alert['location']}")
        print(f"Tweet: {alert['message']}")
        print(f"Link: {alert['url']}")
        print("-" * 80)

        try:
            summary = summarizer(alert["message"], max_length=20, min_length=10, do_sample=False)[0]["summary_text"]
        except:
            summary = alert["message"][:100]

        try:
            sheet.append_row([
                alert["type"],
                alert["location"],
                str(alert["user"]),
                summary,
                alert["url"],
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ])
            print(f"✅ Alert stored: {summary}")
        except Exception as e:
            print(f"❌ Failed to write to sheet: {e}")

    print("🚀 Processing complete.")

# --- Run Example ---
run_emergency_check("#landslide")  # Change hashtag as needed



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 99.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Device set to use cpu
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


🔍 Searching tweets for: #landslide
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
🚨 Rate limit hit for '#landslide'. Waiting 60 seconds...
✅ 10 tweets fetched. Filtering relevant alerts...
⚠️ Alert Detected:
Type: Landslide, Location: Unknown
Tweet: A landslide occurred this morning on National Highway 6 between Seling and Keifang following heavy rainfall. Traffic has been disrupted. Authorities are working to clear the debris and restore movement.

#Mizoram #Landslide #NH

In [ ]:
# --- Install & Imports ---
!pip install --upgrade torch torchvision torchaudio transformers gspread oauth2client spacy
!python -m spacy download en_core_web_sm

import tweepy
import gspread
import spacy
import time
import re
from datetime import datetime
from oauth2client.service_account import ServiceAccountCredentials
from transformers import pipeline

# --- Load NLP Models ---
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
summarizer = pipeline("summarization")
nlp = spacy.load("en_core_web_sm")

# --- Disaster Labels & Keywords ---
CANDIDATE_LABELS = ["accident", "fire", "earthquake", "landslide", "tsunami", "flood"]

DISASTER_KEYWORDS = {
    "fire": ["blaze", "burning", "flames", "wildfire", "smoke", "firefighters"],
    "accident": ["collision", "crash", "injured", "killed", "fatal", "ambulance"],
    "earthquake": ["richter", "tremor", "aftershock", "magnitude", "seismic"],
    "landslide": ["mudslide", "debris", "collapsed", "hill", "mountain", "buried"],
    "tsunami": ["wave", "seawater", "ocean", "quake", "evacuation"],
    "flood": ["rain", "overflow", "submerged", "drowned", "inundated", "storm"]
}

EMERGENCY_HASHTAGS = {
    "#fire": "Fire",
    "#earthquake": "Earthquake",
    "#accident": "Accident",
    "#landslide": "Landslide",
    "#tsunami": "Tsunami",
    "#flood": "Flood"
}

NEGATIVE_CONTEXT_KEYWORDS = [
    "song", "music", "album", "mixtape", "dance", "rap", "party", "track",
    "beat", "concert", "reels", "vibe", "chill", "joint", "smoke", "420", "weed",
    "cigarette", "hookah", "dj", "soundcloud", "studio"
]

# Twitter API credentials
API_KEY = "NmAtlRDhpsWVMXL03tfy8dGgx"
API_SECRET_KEY = "1iWUmP9WlGpCDo0ELagelfNeM6uyK91oFfC6lHmttbYsrVG0s9"
ACCESS_TOKEN = "1921411850429923328-TvE4KKZWuIwifPQUH2WkwdOlZH2PWT"
ACCESS_TOKEN_SECRET = "vdWXSWUG9xVTgsX8rLyMM0IZRgl0GNI4ZQM8DW7wel8wn"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAPq81AEAAAAAl8XbdvHONQb9F61bXqbNk1JeI14%3D0thb5xzDZTXdbT1hnewms1zxHwRld2gSnqMtfUz0XW80jTOfx0"

# --- Google Sheets Credentials ---
GOOGLE_SHEET_JSON_PATH = "/content/twitter-emergncy-project-3789f10635d0.json"
SHEET_NAME = "TWITTER EMERGENCY ALERTS"

# --- Initialize Twitter and Google Clients ---
client = tweepy.Client(bearer_token=BEARER_TOKEN)
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name(GOOGLE_SHEET_JSON_PATH, scope)
sheet = gspread.authorize(creds).open(SHEET_NAME).sheet1

# --- Fetch Tweets Function ---
def fetch_tweets_v2(hashtag, count=10):
    query = f"{hashtag} -is:retweet lang:en"
    try:
        response = client.search_recent_tweets(query=query, max_results=count, tweet_fields=["text", "author_id", "created_at", "id"])
        return [{
            "text": tweet.text,
            "user": tweet.author_id,
            "id": tweet.id
        } for tweet in response.data] if response.data else []
    except tweepy.TooManyRequests:
        print(f"🚨 Rate limit hit for '{hashtag}'. Waiting 60 seconds...")
        time.sleep(60)
        return fetch_tweets_v2(hashtag, count)
    except Exception as e:
        print(f"❌ Error fetching tweets for '{hashtag}':", e)
        return []

# --- Extract Valid Alerts ---
def extract_alerts(tweets, expected_type):
    alerts = []
    keywords = DISASTER_KEYWORDS[expected_type.lower()]
    for tweet in tweets:
        msg = tweet["text"]
        msg_lower = msg.lower()

        if any(neg in msg_lower for neg in NEGATIVE_CONTEXT_KEYWORDS):
            continue

        classification = classifier(msg, CANDIDATE_LABELS)
        top_label = classification["labels"][0]
        confidence = classification["scores"][0]

        if top_label.lower() != expected_type.lower() or confidence < 0.75:
            continue

        if not any(re.search(rf"\b{k}\b", msg_lower) for k in keywords):
            continue

        doc = nlp(msg)
        location = next((ent.text for ent in doc.ents if ent.label_ in ["GPE", "LOC"]), "Unknown")

        alerts.append({
            "message": msg,
            "location": location,
            "type": expected_type,
            "user": tweet["user"],
            "url": f"https://twitter.com/i/web/status/{tweet['id']}"
        })
    return alerts

# --- Main Script to Run ---
def run_emergency_check(test_hashtag):
    test_type = EMERGENCY_HASHTAGS.get(test_hashtag.lower())
    if not test_type:
        print("❌ Invalid hashtag provided.")
        return

    print(f"🔍 Searching tweets for: {test_hashtag}")
    tweets = fetch_tweets_v2(test_hashtag, count=10)

    if not tweets:
        print("❌ No tweets found.")
        return

    print(f"✅ {len(tweets)} tweets fetched. Filtering relevant alerts...")
    all_alerts = extract_alerts(tweets, test_type)

    if not all_alerts:
        print("⚠️ No relevant disaster alerts found.")
        return

    for alert in all_alerts:
        print("⚠️ Alert Detected:")
        print(f"Type: {alert['type']}, Location: {alert['location']}")
        print(f"Tweet: {alert['message']}")
        print(f"Link: {alert['url']}")
        print("-" * 80)

        try:
            summary = summarizer(alert["message"], max_length=20, min_length=10, do_sample=False)[0]["summary_text"]
        except:
            summary = alert["message"][:100]

        try:
            sheet.append_row([
                alert["type"],
                alert["location"],
                str(alert["user"]),
                summary,
                alert["url"],
                datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            ])
            print(f"✅ Alert stored: {summary}")
        except Exception as e:
            print(f"❌ Failed to write to sheet: {e}")

    print("🚀 Processing complete.")

# --- Run Example ---
run_emergency_check("#earthquake")  # Change hashtag as needed



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Device set to use cpu
No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


🔍 Searching tweets for: #earthquake
🚨 Rate limit hit for '#earthquake'. Waiting 60 seconds...
🚨 Rate limit hit for '#earthquake'. Waiting 60 seconds...
🚨 Rate limit hit for '#earthquake'. Waiting 60 seconds...
🚨 Rate limit hit for '#earthquake'. Waiting 60 seconds...
🚨 Rate limit hit for '#earthquake'. Waiting 60 seconds...
🚨 Rate limit hit for '#earthquake'. Waiting 60 seconds...
🚨 Rate limit hit for '#earthquake'. Waiting 60 seconds...
✅ 10 tweets fetched. Filtering relevant alerts...
⚠️ Alert Detected:
Type: Earthquake, Location: United States
Tweet: 2.3 magnitude #earthquake. 64 km from #Mentone, TX, United States https://t.co/SV7tGIaWDo
Link: https://twitter.com/i/web/status/1921542782352584850
--------------------------------------------------------------------------------
✅ Alert stored:  2.3 magnitude #earthquake. 64 km from #Mentone,
⚠️ Alert Detected:
Type: Earthquake, Location: Blangpidie
Tweet: A magnitude 6.2 earthquake took place 16km NNW of Blangpidie, Indonesia at 08:57

In [ ]:
import os
import time
import threading
import random
import string

# Generate random subdomain name
def random_subdomain(length=10):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))

subdomain = random_subdomain()

# Start Streamlit in a thread
def run_streamlit():
    os.system('streamlit run app.py')

threading.Thread(target=run_streamlit).start()
time.sleep(5)  # Let Streamlit start

# Print the subdomain to inform user
print(f"🌐 Starting tunnel with subdomain: {subdomain}")

# Now run localtunnel in the notebook so it shows the link in output
!yes | npx localtunnel --port 8501 --subdomain $subdomain


🌐 Starting tunnel with subdomain: fuy3vg31sc
⠙⠹⠸⠼⠴your url is: https://fuy3vg31sc.loca.lt


In [ ]:
!pip install --upgrade torch torchvision torchaudio transformers gspread oauth2client spacy streamlit
!python -m spacy download en_core_web_sm
!pip install -q streamlit
!npm install -g localtunnel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 62.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
changed 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇

In [ ]:
import os
import time
import threading
import random
import string

# Generate a random subdomain
def random_subdomain(length=10):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))

subdomain = random_subdomain()

# Function to run Streamlit
def run_streamlit():
    os.system('streamlit run app.py --server.port 8501 --server.enableCORS false')

# Start Streamlit in a background thread
threading.Thread(target=run_streamlit).start()

# Wait for server to start
time.sleep(7)

# Start localtunnel
print(f"🌐 Starting LocalTunnel at: https://{subdomain}.loca.lt")
!npx localtunnel --port 8501 --subdomain $subdomain


🌐 Starting LocalTunnel at: https://4laezy46zn.loca.lt
⠙⠹⠸⠼⠴⠦your url is: https://4laezy46zn.loca.lt
^C


In [ ]:
import os
import time
import threading
import random
import string

# Step 1: Generate random subdomain name
def random_subdomain(length=10):
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))

subdomain = random_subdomain()

# Step 2: Write your Streamlit app code if needed
# with open("app.py", "w") as f:
#     f.write("put your streamlit code here")

# Step 3: Start Streamlit in a background thread
def run_streamlit():
    os.system('streamlit run app.py --server.port 8501 --server.enableCORS false')

threading.Thread(target=run_streamlit).start()
time.sleep(5)

# Step 4: Generate tunnel password using curl
print("🔐 Generating tunnel password...")
!curl https://loca.lt/mytunnelpassword

# Step 5: Start tunnel with localtunnel
print(f"🌐 Starting LocalTunnel with subdomain: {subdomain}")
!npx localtunnel --port 8501 --subdomain $subdomain

🔐 Generating tunnel password...
34.57.59.244🌐 Starting LocalTunnel with subdomain: dtpjxigaqu
⠙⠹⠸⠼⠴⠦your url is: https://dtpjxigaqu.loca.lt
